In [2]:
import mediapipe as mp
import cv2

mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# For webcam input:
cap = cv2.VideoCapture(0)

objc[2047]: Class CaptureDelegate is implemented in both /Users/jefflai/Intellidesk/Posture-analysis-system-using-MediaPipe-Pose/myenv/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x15c9725e0) and /Users/jefflai/Intellidesk/Posture-analysis-system-using-MediaPipe-Pose/myenv/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x17abb4860). One of the two will be used. Which one is undefined.
objc[2047]: Class CVWindow is implemented in both /Users/jefflai/Intellidesk/Posture-analysis-system-using-MediaPipe-Pose/myenv/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x15c972630) and /Users/jefflai/Intellidesk/Posture-analysis-system-using-MediaPipe-Pose/myenv/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x16d6f0a68). One of the two will be used. Which one is undefined.
objc[2047]: Class CVView is implemented in both /Users/jefflai/Intellidesk/Posture-analysis-system-using-MediaPipe-Pose/myenv/lib/python3.10/site-packages/cv2/cv2.

In [4]:
def is_ok_gesture(hand_landmarks, image_width, image_height):
    thumb_tip = hand_landmarks.landmark[4]
    index_tip = hand_landmarks.landmark[8]

    # Calculate the Euclidean distance between thumb tip and index fingertip
    distance = ((thumb_tip.x - index_tip.x) ** 2 + (thumb_tip.y - index_tip.y) ** 2) ** 0.5

    # Normalize the distance by the image dimensions
    normalized_distance = distance / (image_width ** 2 + image_height ** 2) ** 0.5

    # Check if the distance is below a certain threshold to consider them as touching
    if normalized_distance < 0.05:  # Threshold, you might need to adjust this based on testing
        # Additional checks for other fingers can be added here
        return True
    return False

def height_gesture(hand_landmarks,image_width, image_height):
    # Get the landmarks for the fingertips, MCP joints, and wrist
    wrist = hand_landmarks.landmark[0]
    thumb_tip = hand_landmarks.landmark[4]
    thumb_mcp = hand_landmarks.landmark[2]
    index_tip = hand_landmarks.landmark[8]
    index_mcp = hand_landmarks.landmark[5]
    middle_tip = hand_landmarks.landmark[12]
    middle_mcp = hand_landmarks.landmark[9]
    ring_tip = hand_landmarks.landmark[16]
    ring_mcp = hand_landmarks.landmark[13]
    pinky_tip = hand_landmarks.landmark[20]
    pinky_mcp = hand_landmarks.landmark[17]

    distance = ((thumb_tip.x - index_tip.x) ** 2 + (thumb_tip.y - index_tip.y) ** 2) ** 0.5

    normalized_distance = distance / (image_width ** 2 + image_height ** 2) ** 0.5

    # Check if the distance is below a certain threshold to consider them as touching
    if normalized_distance < 0.05:  # Threshold, you might need to adjust this based on testing
        # Additional checks for other fingers can be added here
        if (thumb_tip.y < thumb_mcp.y and
        index_tip.y < index_mcp.y and
        middle_tip.y < middle_mcp.y and
        ring_tip.y < ring_mcp.y and
        pinky_tip.y < pinky_mcp.y):
            return "UP"
        if (thumb_tip.y > thumb_mcp.y and
        index_tip.y > index_mcp.y and
        middle_tip.y > middle_mcp.y and
        ring_tip.y > ring_mcp.y and
        pinky_tip.y > pinky_mcp.y):
            return "DOWN"
        
    return None



def table_height(hand_landmarks, image_width, image_height):
    # Get the landmarks for the fingertips, MCP joints, and wrist
    index_tip_y = hand_landmarks.landmark[8].y * image_height
    index_mcp_y = hand_landmarks.landmark[5].y * image_height
    middle_tip_y = hand_landmarks.landmark[12].y * image_height
    middle_mcp_y = hand_landmarks.landmark[9].y * image_height
    ring_tip_y = hand_landmarks.landmark[16].y * image_height
    ring_mcp_y = hand_landmarks.landmark[13].y * image_height
    pinky_tip_y = hand_landmarks.landmark[20].y * image_height
    pinky_mcp_y = hand_landmarks.landmark[17].y * image_height
    thumb_tip_x = hand_landmarks.landmark[4].x * image_width
    thumb_mcp_x = hand_landmarks.landmark[2].x * image_width

    # Check for a fist gesture
    if (index_tip_y > index_mcp_y and
        middle_tip_y > middle_mcp_y and
        ring_tip_y > ring_mcp_y and
        pinky_tip_y > pinky_mcp_y ): 
        return "Fist"
    # Check for gesture 1
    elif index_tip_y < index_mcp_y and middle_tip_y > middle_mcp_y and ring_tip_y > ring_mcp_y and pinky_tip_y > pinky_mcp_y:
        return "Gesture 1"
    # Check for gesture 2
    elif index_tip_y < index_mcp_y and middle_tip_y < middle_mcp_y and ring_tip_y > ring_mcp_y and pinky_tip_y > pinky_mcp_y:
        return "Gesture 2"
    # Check for gesture 3
    elif index_tip_y < index_mcp_y and middle_tip_y < middle_mcp_y and ring_tip_y < ring_mcp_y and pinky_tip_y > pinky_mcp_y:
        return "Gesture 3"
    else:
        return "Unknown Gesture"






In [5]:

import time
import cv2

# Replace this with the actual initialization of your video capture and hand tracking
cap = cv2.VideoCapture(0)  # or whatever your source is
# hands = SomeHandTrackingModule()  # initialize your hand tracking module here

gesture_start_times = {'Gesture 1': None, 'Gesture 2': None, 'Gesture 3': None}
fist_signal_sent = False
current_gesture = None
gesture_threshold = 3  # seconds

while cap.isOpened():
    success, image = cap.read()
    if not success:
        continue

    image_height, image_width, _ = image.shape

    # Flip the image horizontally for a later selfie-view display
    # Convert the BGR image to RGB
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)

    # Process the image and detect hands
    results = hands.process(image)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Assuming table_height function returns 'Gesture 1', 'Gesture 2', 'Gesture 3', or 'Fist'
            detected_gesture = table_height(hand_landmarks, image_height, image_width)

            # Check if the detected gesture is one we're tracking
            if detected_gesture in gesture_start_times:
                if current_gesture != detected_gesture:
                    # New gesture detected, start timing
                    current_gesture = detected_gesture
                    gesture_start_times[detected_gesture] = time.time()
                    fist_signal_sent = False  # Reset the fist signal
                else:
                    # Gesture is sustained, check duration
                    if time.time() - gesture_start_times[detected_gesture] >= gesture_threshold:
                        # Send a signal to hold a fist if it wasn't sent already
                        if not fist_signal_sent:
                            print(f"Signal sent for {detected_gesture}: Hold a fist to confirm")
                            fist_signal_sent = True
            elif detected_gesture == 'Fist' and fist_signal_sent:
                # Fist detected after signal, confirm the posture change
                print(f"Change table posture to {current_gesture}")
                # Reset the gesture and fist signal
                current_gesture = None
                fist_signal_sent = False
                for gesture in gesture_start_times:
                    gesture_start_times[gesture] = None
            else:
                # Different gesture or no gesture detected, reset the current gesture and timer for the previous gesture
                current_gesture = None
                fist_signal_sent = False
                for gesture in gesture_start_times:
                    gesture_start_times[gesture] = None

    # Add your other processing here
    # ...

    # Show the processed image - convert it back to BGR to display with OpenCV
    cv2.imshow('Hand Tracking', cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


2023-12-21 15:43:13.170 Python[2047:24739] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


Signal sent for Gesture 2: Hold a fist to confirm
Change table posture to Gesture 2
Signal sent for Gesture 1: Hold a fist to confirm
Change table posture to Gesture 1
Signal sent for Gesture 3: Hold a fist to confirm


KeyboardInterrupt: 